In [1]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from tqdm import tqdm
import os

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
from lyricsgenius import Genius

In [3]:
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Genius config

In [4]:
load_dotenv()

True

In [5]:
token = os.getenv("genius-token")
genius = Genius(token)

# Importing pickle file

In [6]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [7]:
tracks.head()

,url,artist_clean,track_clean
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [8]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.reset_index(inplace=True)
tracks.head()

,index,url,artist,track
0,8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# Selenium

Configuro mi pestaña de selenium

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://chromedriver.chromium.org/downloads')

Voy a probar que me funcione la búsqueda de un elemento

In [ ]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## extraigo la información de la letra de cada canción
lyrics = driver.find_element('css selector', '#lyrics-root > div:nth-child(3)').text
lyrics2 = driver.find_element('css selector', '#lyrics-root > div:nth-child(6)').text
sleep(1)
release_date = driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(2) > div').text
#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.kEkpYM > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(3) > div

ESTOY TRABAJANDO AQUÍ
Clicando el read more

In [ ]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## clico el all credits
driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div.HeaderMetadata__Section-sc-1p42fnf-2.glevmK > div > a').click()
sleep(1)

CONTINUO AQUÍ CON B4S

In [ ]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## clico el all credits
driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div.HeaderMetadata__Section-sc-1p42fnf-2.glevmK > div > a').click()
sleep(1)
page_source = driver.page_source
url = driver.current_url
res = requests.get(url + '#song-info')
soup = BeautifulSoup(res.content, 'html.parser')
pagina = soup.text

In [ ]:
pagina

In [ ]:
release_date

In [ ]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [ ]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre del actor que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    all_lyrics = ''
    for item in range(1,15):
        try:
            lyrics = driver.find_element('css selector', f'#lyrics-root > div:nth-child({item})').text
            if len(lyrics) > 12:
                all_lyrics += lyrics
            else:
                continue
        except:
            continue
    canciones['lyrics'].append(all_lyrics)
    sleep(1)
    try:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[3]/div[1]/div[2]/div/div[1]/span[1]/span').text
    except:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div').text
    canciones['release_date'].append(release_date)

In [ ]:
canciones

# Beautiful Soup
Prueba

In [ ]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [ ]:
url = 'https://genius.com/Drake-one-dance-lyrics#song-info'

In [ ]:
res = requests.get(url)
res ## tiene que dar 200 

In [ ]:
type(res.content) ## para consultarel html de la página

In [ ]:
sopa = BeautifulSoup(res.content, 'html.parser')
type(sopa) ## es más amigable, pero sigue siendo html

Datos canciones

In [ ]:
pagina = sopa.text
re.findall(r'Date(\w+ \d+, \d+)', pagina)[0]

In [ ]:
pagina

# Beautiful soup + selenium

In [ ]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [ ]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre de canción que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    page_source = driver.page_source
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    pagina = sopa.text
    canciones['release_date'].append(re.findall(r'Date(\w+ \d+, \d+)', pagina)[0])
    #break

In [ ]:
tracks.iloc[51,:]

Aquí con selenium

In [ ]:
almacen_paginas = {
    'pagina' : [], 
    'lyrics':[],
    'featuring' : [],
    'produced_by': [],
    'label': [],
    'written by': [],
    'drums': [],
    'guitar': [],
    'recorded_at' : [],
    'release_date':[],
    'url':[]
}

In [9]:
with open('../data/pickle/genius_dataset_scraped15.pickle', 'rb') as almacen_paginas:
    almacen_paginas = pickle.load(almacen_paginas)

In [11]:
#test = tracks.iloc[(864):,:].reset_index(drop=True)
test = tracks.iloc[(len(almacen_paginas['url'])):,:].reset_index(drop=True)
filename = 'genius_dataset_scraped16'
test.head()

,index,url,artist,track
0,3661068,https://open.spotify.com/track/3gHLXLJ9JE1oOay...,meek mill,100 summers
1,3661069,https://open.spotify.com/track/3nTDO6k4R6i3FpK...,ski mask the slump god,so high
2,3661071,https://open.spotify.com/track/5Ber68jZ7ytegr2...,michael bublé,santa claus is coming to town
3,3661072,https://open.spotify.com/track/6CTWathupIiDs7U...,ella mai,trip
4,3661075,https://open.spotify.com/track/1Bp5LdIxR3XlprI...,ski mask the slump god,adults swim


In [10]:
#almacen_paginas['pagina'].append(np.nan)
#almacen_paginas['lyrics'].append(np.nan)
#almacen_paginas['url'].pop()
print(len(almacen_paginas['lyrics']), len(almacen_paginas['pagina']), len(almacen_paginas['url']))

2876 2876 2876


In [12]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
for index, row in test.iterrows():
    print(index, row['artist'].split(',')[0],'---', row['track'].split('-')[0])
    almacen_paginas['url'].append(row['url'])
    if index == 0: 
        driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
        sleep(1)
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
        sleep(1)
    else:
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#sticky-nav > div.StickyNavdesktop__Right-sc-60o25r-2.kMDkxm > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
        sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    try:
        driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
        driver.implicitly_wait(30)
        ## clico el all credits
        try:
            driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[4]/div/a').click()
        except:
            try:
                driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div/a').click()
            except:
                try:
                    driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[2]/div/a').click()
                except:
                    try:
                        driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[1]/div/a').click()
                    except:
                        print('oye que no va')
                        break
        #'//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div/div/a'
        sleep(1)
        page_source = driver.page_source
        url = driver.current_url
        res = requests.get(url + '#song-info')
        soup = BeautifulSoup(res.content, 'html.parser')
        pagina = soup.text
        almacen_paginas['pagina'].append(pagina)
        try:
            almacen_paginas['lyrics'].append(re.findall(r'Lyrics(.*)EmbedCancel?', pagina)[0])
        except:
            almacen_paginas['lyrics'].append(np.nan)
        with open(f'../data/pickle/{filename}.pickle', 'wb') as data_scrapeado:
            pickle.dump(almacen_paginas, data_scrapeado)
        clear_output(wait=True)
    except:
        almacen_paginas['pagina'].append(np.nan)
        almacen_paginas['lyrics'].append(np.nan)
        clear_output(wait=True)

113 metro boomin --- only you (feat. wizkid, offset & j balvin)


In [ ]:
almacen_paginas['lyrics'][122]

In [ ]:
almacen_paginas['pagina'][0]

In [ ]:
#almacen_paginas['pagina'][0]
print(re.findall(r'Lyrics(.*)EmbedCancel?', pagina)[0])

In [ ]:
#almacen_paginas['pagina'][1]
print('label', re.findall(r'(.*)', almacen_paginas['pagina'][1]))

In [ ]:
for pagina in almacen_paginas['pagina']:
    #print(pagina)
    try:
        print('producción', re.findall(r'Produced By(.*)Written By?', pagina), 'chachi')
        #print(re.findall(r'Licensing(.*)[Distributor|Guest Appearances|Licensing]?', pagina))
        print('label', re.findall(r'Label(.*)Publisher|Licensing|Premiere', pagina))
        print('date', re.findall(r'Release Date(\w+ \d+, \d+)', pagina), 'chachi')
    except:
        try:
            print(re.findall(r'Produced By?(.*)?Written By?(.*)?Primary Artist?(.*)?Publisher?(.*)Mixing Engineer?(.*)?Release Date(.*)', pagina))
        except:
            pass
    #print(re.findall(r'Produced By?(.*)?Written By?(.*)?Keyboards?(.*)?Video Director?(.*)?Label?(.*)?Vocals?(.*)?Bass Guitar?(.*)?Drums?(.*)?Guitar?(.*)?Recorded At?(.*)?Release Date?(.*)?Songs That?', pagina))

In [ ]:
re.findall(r'Produced By?(.*)?Written By?(.*)?Primary Artist?(.*)?Publisher?(.*)Mixing Engineer?(.*)?Release Date(.*)', almacen_paginas['pagina'][-2])

In [ ]:
almacen_paginas['pagina'][-2]

In [ ]:
re.findall(r'Produced By(.*)Written By(.*)Keyboards(.*)Video Director(.*)Label(.*)Vocals(.*)Bass Guitar(.*)Drums(.*)Guitar(.*)Recorded At(.*)Release Date(.*)Songs That', pagina)
#Lyrics(.*)Produced By(.*)Written By(.*)Keyboards(.*)Video Director(.*)Label(.*)Vocals(.*)Bass Guitar(.*)Drums(.*)Guitar(.*)Recorded At(.*)Release Date(.*)Songs', almacen_paginas['pagina'][-1])

In [ ]:
re.findall(r'Lyrics(.*)', pagina)[1]
#Lyrics(.*)Produced By(.*)Written By(.*)Keyboards(.*)Video Director(.*)Label(.*)Vocals(.*)Bass Guitar(.*)Drums(.*)Guitar(.*)Recorded At(.*)Release Date(.*)Songs', almacen_paginas['pagina'][-1])

In [ ]:
re.findall(r'Featuring(\w+\s+?\w+?)Produced', almacen_paginas['pagina'][-1])

In [ ]:
re.findall(r'Produced By(.*)Written By', almacen_paginas['pagina'][-1])

In [ ]:
re.findall(r'Written By(.*)Keyboards', almacen_paginas['pagina'][-1])

In [ ]:
re.findall(r'Featuring(.*)', almacen_paginas['pagina'][-1])[1]

In [ ]:
tracks

In [ ]:
urls = []
for index, row in tracks.iterrows():
    urls.append(f"https://genius.com/{row['artist'].split(',')[0].replace(' ','-')}-{row['track'].replace(' ','-')}")
urls[10:15]

# test

In [ ]:
genius_search_url = f"http://api.genius.com/search?q={'drake one love'}&access_token={token}"

In [ ]:
response = requests.get(genius_search_url)
json_data = response.json()

In [ ]:
json_data

# Export

In [ ]:
with open('../data/pickle/genius_dataset.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)